In [1]:
import findspark
#findspark.init('/usr/hdp/current/spark2-client')
findspark.init()
findspark.find()

'C:\\Spark\\spark-hadoop'

In [2]:
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]")\
.config("spark.jars.packages","mysql:mysql-connector-java:5.1.6,org.xerial:sqlite-jdbc:3.23.1,org.mongodb.spark:mongo-spark-connector_2.11:2.3.0,com.datastax.spark:spark-cassandra-connector_2.11:2.3.1")\
.config("spark.mongodb.input.uri","mongodb://127.0.0.1/SampleCollections.Sakila_customers")\
.config("spark.mongodb.output.uri","mongodb://127.0.0.1/SampleCollections.Sakila_customers")\
.appName("ch09DataSources_1").getOrCreate()

In [3]:
# spark.conf.set("spark.mongodb.input.uri","mongodb://127.0.0.1/SampleCollections.Sakila_customers")
# spark.conf.set("spark.mongodb.output.uri","mongodb://127.0.0.1/SampleCollections.Sakila_customers")

In [4]:
# import pyspark
# from pyspark.sql import SparkSession
# spark = SparkSession.builder.master("yarn")\
# .config("spark.jars","/home/kranthidr/softwares/jars/mysql-connector-java-5.1.6.jar")\
# .appName("ch09DataSources_1").getOrCreate()


# "org.xerial:sqlite-jdbc:3.23.1"
# "org.mongodb.spark:mongo-spark-connector_2.11:2.3.0"
# "com.datastax.spark:spark-cassandra-connector_2.11:2.3.1"


In [5]:
sc = spark.sparkContext

In [6]:
for x in sc._conf.getAll():
    if '/proxy/' in x[1]:
        print(x[1])

In [7]:
spark

In [8]:
spark.conf.get("spark.mongodb.input.uri")

'mongodb://127.0.0.1/SampleCollections.Sakila_customers'

In [9]:
spark.conf.get("spark.mongodb.output.uri")

'mongodb://127.0.0.1/SampleCollections.Sakila_customers'

In [10]:
mongo_df = spark.read.format("com.mongodb.spark.sql.DefaultSource").load()

In [11]:
mongo_df.printSchema()

root
 |-- Address: string (nullable = true)
 |-- City: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- District: string (nullable = true)
 |-- First Name: string (nullable = true)
 |-- Last Name: string (nullable = true)
 |-- Phone: string (nullable = true)
 |-- Rentals: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- Film Title: string (nullable = true)
 |    |    |-- Payments: array (nullable = true)
 |    |    |    |-- element: struct (containsNull = true)
 |    |    |    |    |-- Amount: double (nullable = true)
 |    |    |    |    |-- Payment Date: string (nullable = true)
 |    |    |    |    |-- Payment Id: integer (nullable = true)
 |    |    |-- Rental Date: string (nullable = true)
 |    |    |-- Return Date: string (nullable = true)
 |    |    |-- filmId: integer (nullable = true)
 |    |    |-- rentalId: integer (nullable = true)
 |    |    |-- staffId: integer (nullable = true)
 |-- _id: integer (nullable = true)

In [12]:
mongo_df.count()

599

In [13]:
mongo_df.show(10)

+--------------------+--------------+-------------+--------------+----------+---------+------------+--------------------+---+
|             Address|          City|      Country|      District|First Name|Last Name|       Phone|             Rentals|_id|
+--------------------+--------------+-------------+--------------+----------+---------+------------+--------------------+---+
|      1913 Hanoi Way|        Sasebo|        Japan|      Nagasaki|      MARY|    SMITH| 28303384290|[[MUSKETEERS WAIT...|  1|
|    1121 Loja Avenue|San Bernardino|United States|    California|  PATRICIA|  JOHNSON|838635286649|[[DOORS PRESIDENT...|  2|
|   692 Joliet Street|       Athenai|       Greece|        Attika|     LINDA| WILLIAMS|448477190408|[[RINGS HEARTBREA...|  3|
|    1566 Inegl Manor|      Myingyan|      Myanmar|      Mandalay|   BARBARA|    JONES|705814003527|[[BEDAZZLED MARRI...|  4|
|     53 Idfu Parkway|        Nantou|       Taiwan|        Nantou| ELIZABETH|    BROWN| 10655648674|[[BETRAYED REAR, .

In [6]:
# COMMAND ----------
mysql_url = "jdbc:mysql://nn01.itversity.com/retail_db"
mysql_driver = "com.mysql.jdbc.Driver"
mysql_dbtable = "categories"
mysql_user = "retail_dba"
mysql_password = "itversity"

In [7]:
dataframe_mysql = spark.read.format("jdbc")\
.option("url", mysql_url)\
.option("dbtable",mysql_dbtable)\
.option("driver",mysql_driver)\
.option("user", mysql_user)\
.option("password", mysql_password).load()

In [8]:
dataframe_mysql.show()

+-----------+----------------------+-------------------+
|category_id|category_department_id|      category_name|
+-----------+----------------------+-------------------+
|          1|                     2|           Football|
|          2|                     2|             Soccer|
|          3|                     2|Baseball & Softball|
|          4|                     2|         Basketball|
|          5|                     2|           Lacrosse|
|          6|                     2|   Tennis & Racquet|
|          7|                     2|             Hockey|
|          8|                     2|        More Sports|
|          9|                     3|   Cardio Equipment|
|         10|                     3|  Strength Training|
|         11|                     3|Fitness Accessories|
|         12|                     3|       Boxing & MMA|
|         13|                     3|        Electronics|
|         14|                     3|     Yoga & Pilates|
|         15|                  

In [69]:
# COMMAND ----------
spark.conf.set("spark.cassandra.connection.host", "wn05.itversity.com")

In [70]:
from pyspark.sql import SQLContext
sc = spark.sparkContext
sqlContext = SQLContext(sc)

In [78]:
spark.read.format("org.apache.spark.sql.cassandra").options(table="orders", keyspace="raj").load().show()

+--------+-----------------+--------------------+---------------+
|order_id|order_customer_id|          order_date|   order_status|
+--------+-----------------+--------------------+---------------+
|   12300|              127|2013-10-08 00:00:...|PENDING_PAYMENT|
|   52271|             3332|2014-06-18 00:00:...|         CLOSED|
|   36223|             1020|2014-03-04 00:00:...|        PENDING|
|   46440|            11945|2014-05-09 00:00:...|       COMPLETE|
|   16058|             9182|2013-11-03 00:00:...|         CLOSED|
|   48208|             9723|2014-05-20 00:00:...|     PROCESSING|
|   67948|             2071|2013-12-26 00:00:...|         CLOSED|
|   30697|             8412|2014-01-31 00:00:...|        ON_HOLD|
|    4016|            11946|2013-08-17 00:00:...|PENDING_PAYMENT|
|   68247|             2435|2014-03-19 00:00:...|         CLOSED|
|   52486|             7946|2014-06-19 00:00:...|       COMPLETE|
|    1006|             9028|2013-07-30 00:00:...|       COMPLETE|
|   16751|